# Install

In [ ]:
import requests
import pandas as pd
!pip install textblob
from textblob import TextBlob  # Türkçe için ayrı öneri de vereceğim
pip install streamlit


# Main Script

In [ ]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("YOUTUBE_API_KEY")
VIDEO_ID = "pcGzWN1HEzU"

def fetch_comments(video_id, api_key):
    comments = []
    page_token = ''
    
    while True:
        url = "https://www.googleapis.com/youtube/v3/commentThreads"
        params = {
            "part": "snippet",
            "videoId": video_id,
            "key": api_key,
            "maxResults": 100,
            "textFormat": "plainText",
            "pageToken": page_token
        }

        response = requests.get(url, params=params)
        data = response.json()

        if 'items' not in data:
            print("Error:", data)
            break

        for item in data['items']:
            snippet = item['snippet']['topLevelComment']['snippet']
            comments.append(snippet['textDisplay'])

        page_token = data.get('nextPageToken')
        if not page_token:
            break

    df = pd.DataFrame(comments, columns=["comment"])
    df.to_csv("comments.csv", index=False)
    return df


# Sentiment Analysis

In [ ]:
from textblob import TextBlob

def analyze_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    if polarity > 0:
        return "positive"
    elif polarity < 0:
        return "negative"
    else:
        return "neutral"


# Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

def plot_sentiment_distribution(df):
    sns.countplot(data=df, x="sentiment")
    plt.title("Sentiment Distribution")
    plt.xlabel("Sentiment")
    plt.ylabel("Count")
    plt.show()

def generate_wordcloud(df):
    text = " ".join(df["comment"])
    wordcloud = WordCloud(width=800, height=400).generate(text)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title("Word Cloud")
    plt.show()


# Web Interface

In [ ]:
import streamlit as st
import pandas as pd
from main import fetch_comments
from sentiment import analyze_sentiment
from visualization import plot_sentiment_distribution, generate_wordcloud

st.title("🎥 YouTube Comment Analyzer")

video_id = st.text_input("Enter YouTube Video ID:")
if st.button("Fetch & Analyze"):
    with st.spinner("Fetching comments..."):
        df = fetch_comments(video_id, api_key="YOUR_API_KEY_HERE")
        df["sentiment"] = df["comment"].apply(analyze_sentiment)
        st.success("Comments fetched and analyzed!")

        st.subheader("📊 Sentiment Distribution")
        st.bar_chart(df["sentiment"].value_counts())

        st.subheader("☁️ Word Cloud")
        generate_wordcloud(df)

        st.dataframe(df.head())


# Dependencies File

In [ ]:
requests
pandas
textblob
seaborn
matplotlib
wordcloud
streamlit
python-dotenv
